In [7]:
import pandas as pd 
import sqlalchemy
from binance.client import Client
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
api_key, api_secret = "", ""

In [3]:
client = Client(api_key, api_secret)

In [4]:
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')


In [14]:
df = pd.read_sql('BTCUSDT', engine)
df.head()

,symbol,Time,Price
0,BTCUSDT,2023-07-29 10:17:58.627,29270.00
1,BTCUSDT,2023-07-29 10:17:58.655,29269.99
2,BTCUSDT,2023-07-29 10:17:58.655,29269.99
3,BTCUSDT,2023-07-29 10:17:58.655,29269.99
4,BTCUSDT,2023-07-29 10:17:59.979,29270.00


In [18]:
# Trendfollowing - X/Twitter
# if the crypto was rising by x % -> Buy
# exit when profit is above 0.15% or loss is crossing -0.15%
# Lookback defines the number of timesteps to go back to.
# 1 timestep = 1 second

In [ ]:
def strategy(entry, lookback, qty, open_position=False):
    while True:
        df = pd.read_sql('BTCUSDT', engine)
        loockbackperiod = df.iloc[-lookback:]
        cumret = (loockbackperiod.Price.pct_change() +1).cumprod() -1
        if not open_position:
            if cumret[cumret.last_valid_index()] > entry:
                order = client.create_order(symbol='BTCUSDT',
                                            side='BUY',
                                            type='MARKET',
                                            quantity=qty)
                print(order)
                open_position = True
                break
    
    if open_position:
        while True:
            df = pd.read_sql('BTCUSDT', engine)
            sincebuy = df.loc[df.Time > pd.to_datetime(order['transactionTime'],
                                                       unit='ms')]